<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/pose_land_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声なし版

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp

# MediaPipe Poseの初期化
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 動画ファイルの読み込み
input_video_path = '/content/drive/MyDrive/Colab_files/dance-sample.mp4'
cap = cv2.VideoCapture(input_video_path)

# 動画ファイルのプロパティ取得
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 出力動画ファイルの設定
output_video_path = '/content/drive/MyDrive/Colab_files/dance-sample-l.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR画像をRGBに変換
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ポーズランドマークの検出
    results = pose.process(rgb_frame)

    # ランドマークの描画
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # フレームを書き込み
    out.write(frame)

cap.release()
out.release()
pose.close()

print(f'Output video saved to {output_video_path}')

# 音声あり版

In [ ]:
!pip install mediapipe opencv-python moviepy
!sudo apt-get install ffmpeg

In [ ]:
import cv2
import mediapipe as mp
from moviepy.editor import VideoFileClip, AudioFileClip

# MediaPipe Poseの初期化
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 入力動画ファイルのパス
input_video_path = '/content/drive/MyDrive/Colab_files/dance-sample.mp4'

# MoviePyを使って動画と音声を読み込み
clip = VideoFileClip(input_video_path)
audio = clip.audio

# OpenCVを使って動画ファイルを読み込み
cap = cv2.VideoCapture(input_video_path)

# 動画ファイルのプロパティ取得
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 出力動画ファイルの設定
output_video_path = 'output_with_landmarks.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR画像をRGBに変換
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ポーズランドマークの検出
    results = pose.process(rgb_frame)

    # ランドマークの描画
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # フレームを書き込み
    out.write(frame)

cap.release()
out.release()
pose.close()

# MoviePyを使って映像と音声を結合
final_clip = VideoFileClip(output_video_path)
final_clip = final_clip.set_audio(audio)
final_clip.write_videofile("final_output_with_audio.mp4", codec='libx264', audio_codec='aac')

print('Output video saved as final_output_with_audio.mp4')

# ワールド座標のファイル保存を追加

In [ ]:
import cv2
import mediapipe as mp
import csv
from moviepy.editor import VideoFileClip

# MediaPipe Poseの初期化
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 入力動画ファイルのパス
input_video_path = '/content/drive/MyDrive/Colab_files/dance-sample.mp4'

# MoviePyを使って動画と音声を読み込み
clip = VideoFileClip(input_video_path)
audio = clip.audio

# OpenCVを使って動画ファイルを読み込み
cap = cv2.VideoCapture(input_video_path)

# 動画ファイルのプロパティ取得
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 出力動画ファイルの設定
output_video_path = 'output_with_landmarks.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSVファイルの設定
csv_file_path = 'landmarks_world_coordinates.csv'
csv_file = open(csv_file_path, mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'landmark_index', 'x', 'y', 'z', 'visibility'])

frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR画像をRGBに変換
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ポーズランドマークの検出
    results = pose.process(rgb_frame)

    # ランドマークの描画
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # ランドマークのワールド座標を取得
        for i, landmark in enumerate(results.pose_world_landmarks.landmark):
            csv_writer.writerow([frame_index, i, landmark.x, landmark.y, landmark.z, landmark.visibility])

    # フレームを書き込み
    out.write(frame)
    frame_index += 1

cap.release()
out.release()
pose.close()
csv_file.close()

# MoviePyを使って映像と音声を結合
final_clip = VideoFileClip(output_video_path)
final_clip = final_clip.set_audio(audio)
final_clip.write_videofile("final_output_with_audio.mp4", codec='libx264', audio_codec='aac')

print('Output video saved as final_output_with_audio.mp4')
print(f'Landmark world coordinates saved as {csv_file_path}')

# ランドマークを上から見た映像に変換（音声付き）